# Base rewards distribution

## Install dependencies

In [14]:
%%capture
%pip install ../ || pip install git+https://github.com/lidofinance/lido-sandbox.git

## Init locator

Instantiates all Lido contacts

In [2]:
from lido_sandbox.locator import Locator

locator = Locator()
_, lido = locator.lido
_, router = locator.staking_router

## Init Lido

Initializes the Lido contract and put [the stone in the elevator](https://img2.reactor.cc/pics/post/%D0%B2%D1%81%D1%91-%D0%BF%D0%BB%D0%BE%D1%85%D0%BE-%D1%84%D1%8D%D0%BD%D0%B4%D0%BE%D0%BC%D1%8B-%D0%BB%D0%B8%D1%84%D1%82-7197897.jpeg)

In [3]:
lido.balance = 1 * 10**18
lido.initialize()

## Submits from users

In [4]:
%%capture
lido.submit('user 1', 64 * 10**18)
lido.submit('user 2', 64 * 10**18)

## Init staking modules

Adds 2 staking modules to the staking router and adds some operators and keys

In [5]:
router.add_staking_module(name='curated', staking_module_address='curated module', target_share=10000, staking_module_fee=500, treasury_fee=500)
router.add_staking_module(name='community', staking_module_address='community module', target_share=10000, staking_module_fee=700, treasury_fee=300)

operators_per_module = 5
submitted_keys_per_operator = 10

### Curated module

In [6]:
curated_module = router.get_staking_module_instance('curated module')
for no_index in range(operators_per_module):
    curated_module.add_node_operator('curated operator %d' % (no_index), 'curated operator %d' % (no_index))
    curated_module.add_signing_keys(
        no_index, 
        [["curated key %d.%d" % (no_index, key_index), "curated signature %d.%d" % (no_index, key_index)] for key_index in range(submitted_keys_per_operator)]
    )
    curated_module.set_node_operator_staking_limit(no_index, submitted_keys_per_operator)

### Community module

In [7]:
community_module = router.get_staking_module_instance('community module')
for no_index in range(operators_per_module):
    community_module.add_node_operator('community operator %d' % (no_index), 'community operator %d' % (no_index))
    community_module.add_signing_keys(
        no_index, 
        [["community key %d.%d" % (no_index, key_index), "community signature %d.%d" % (no_index, key_index)] for key_index in range(submitted_keys_per_operator)]
    )
    community_module.set_node_operator_staking_limit(no_index, submitted_keys_per_operator)

## Deposit

Performs deposits to the keys

In [8]:
lido.deposit(max_deposits_count=100, staking_module_id=1)
lido.deposit(max_deposits_count=100, staking_module_id=2)

## Oracle report

### Global Lido statistics before the report

In [9]:
from lido_sandbox.helpers import print_global_lido_statistic
print_global_lido_statistic(locator)

total supply,total shares,buffered,cl balance,cl vals,deposited vals
129.0,129.0,1.0,0.0,0,4


### Stakers before the report

In [10]:
from lido_sandbox.helpers import print_lido_stakers
print_lido_stakers(locator)

address,shares,steth
0xdead,1.0,1.0
user 1,64.0,64.0
user 2,64.0,64.0


### The main phase of the oracle report
Reports deposited validators and some rewards

In [11]:
%%capture

from time import time
current_timestamp = int(time())

lido.handle_oracle_report(
    report_timestamp = current_timestamp,
    time_elapsed = current_timestamp - lido._last_report_timestamp,
    cl_validators = lido._deposited_validators,
    cl_balance = lido._deposited_validators * lido.DEPOSIT_SIZE,
    withdrawal_vault_balance = 2 * 10 ** 18,
    el_rewards_vault_balance = 2 * 10 ** 18,
    shares_requested_to_burn = 0,
    withdrawal_finalization_batches = [],
    simulated_share_rate = 0
)

### Global Lido statistics after the report

In [12]:
from lido_sandbox.helpers import print_global_lido_statistic
print_global_lido_statistic(locator)

total supply,total shares,buffered,cl balance,cl vals,deposited vals
129.0967,129.0097,1.0967,128.0,4,4


### Stakers after the report

In [13]:
from lido_sandbox.helpers import print_lido_stakers
print_lido_stakers(locator)

address,shares,steth
0xdead,1.0,1.0007
user 1,64.0,64.0432
user 2,64.0,64.0432
lido,0.0,0.0
curated module,0.0024,0.0024
community module,0.0034,0.0034
treasury,0.0039,0.0039
